In [2]:
#Processing evcc block ack
import csv
import numpy as np

with open('max_evcc_block_ack.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    startint_seq_num_block_ack = []
    timelist_block_ack = []
    
    for row in rows:
        startint_seq_num_block_ack.append(int(row['Starting Sequence Number']))
        timelist_block_ack.append(float(row['Time']))

In [3]:
#for debug
if(len(startint_seq_num_block_ack) != len(timelist_block_ack)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_block_ack))

4852


In [4]:
#Processing evcc send
with open('max_evcc_send.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    timelist_evcc_send = []
    seq_num_evcc_send = []
    data_length_evcc_send = []
    info_evcc_send = []
    frame_num_evcc_send = []
    
    for row in rows:
        timelist_evcc_send.append(float(row['Time']))
        seq_num_evcc_send.append(int(row['Sequence number']))
        data_length_evcc_send.append(int(row['Data Length']))
        info_evcc_send.append(str(row['Info']))
        frame_num_evcc_send.append(int(row['No.']))

In [5]:
#for debug
if(len(timelist_evcc_send) != len(seq_num_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(data_length_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(info_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(frame_num_evcc_send)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_evcc_send))

2435


In [6]:
timelist_evcc_send[0]

48.775482

In [7]:
timelist_evcc_send_split = []
seq_num_evcc_send_split = []
data_length_evcc_send_split = []
info_evcc_send_split = []
frame_num_evcc_send_split = []

round_start_point = 0
round_end_point = 0

sdpreq_index_list = []
#Fetch first packet index of data length = 74 (SDP request) in each round
for i in range(0, len(data_length_evcc_send)):
    if(i == 0 and data_length_evcc_send[i] == 74):
        sdpreq_index_list.append(i)
    if(i > 0 and data_length_evcc_send[i] == 74 and data_length_evcc_send[i-1] != 74):
        sdpreq_index_list.append(i)
sdpreq_index_list.append(len(data_length_evcc_send)) #Add the end of list index

for i in range(0, len(sdpreq_index_list)-1):
    timelist_evcc_send_split.append([timelist_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    seq_num_evcc_send_split.append([seq_num_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    data_length_evcc_send_split.append([data_length_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    info_evcc_send_split.append([info_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    frame_num_evcc_send_split.append([frame_num_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])

In [8]:
#for debug
if(len(timelist_evcc_send_split) != len(seq_num_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(data_length_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(info_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(frame_num_evcc_send_split)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_evcc_send_split)) #Output # of testing rounds

20


In [9]:
#for debug, this is ideally the time of each testing round
for i in range(1, len(timelist_evcc_send_split)):
    print(timelist_evcc_send_split[i][0]-timelist_evcc_send_split[i-1][0])

31.261074000000008
30.812400999999994
29.97750599999999
30.32952400000002
30.300457999999992
30.217693999999995
30.51589400000003
30.130146999999965
31.149827000000016
30.86987899999997
30.029340000000047
30.55733299999997
29.59714200000002
29.969460999999967
29.978530000000035
30.327392999999972
29.277684000000022
30.58722899999998
30.305931999999984


In [ ]:
#Set evcc time anchor in each round per stage
evcc_send_time_anchor = []
#data_length_in_each_stage = [74, 100, 189, 173, 173, 173, 185, 461, 189, 173]
wierd_packet = {}
packet_loss_testing_round_3_flag = False
packet_loss_testing_round_3_flag_2 = False
packet_loss_testing_round_5_flag_1 = False
packet_loss_testing_round_5_flag_2 = False
packet_loss_testing_round_5_flag_3 = False
packet_loss_testing_round_17_flag = False
packet_loss_testing_round_17_flag_2 = False

def printinfo(stageNum, frameNum, info_string):
    print("-----------------------------------------")
    print("Stage " + str(stageNum) + " finish. No. " + str(frameNum) + ", Info " + str(info_string))

def print_time_difference(evcc_send_time_anchor_each_round):
    print("Time difference: " + str(evcc_send_time_anchor_each_round[len(evcc_send_time_anchor_each_round)-1]-
                          evcc_send_time_anchor_each_round[len(evcc_send_time_anchor_each_round)-2]))
    
def checkstage_num(stageNum, valid_stageNum, info_string, time_string, datalen, seqnum, frameNum):
    if(stageNum == valid_stageNum):
        check_wield_packet_list(frameNum)
        return True
    else:
        #debug
        #print("\nWhat is this packet? Data Length = " + str(datalen) + " No. " + str(frameNum))
        #print("Time " + str(time_string) + " #Seq " + str(seqnum) + " Info " + str(info_string) + "\n")
        wierd_packet[frameNum] = [datalen, stageNum]
        return False

def check_wield_packet_list(frameNum):
    if(wierd_packet.get(frameNum) != None):
        del wierd_packet[frameNum]
        print("\nNo. " + str(frameNum) + " has been found. Delete from wierd list.\n")
    
for i in range(0, len(data_length_evcc_send_split)):
#for i in range(5, 6):
    print("\n-------- Test Round " + str(i+1) + " --------")
    #Stage 1
    evcc_send_time_anchor_each_round = [timelist_evcc_send_split[i][0]]
    stageNum = 1
    printinfo(stageNum, frame_num_evcc_send_split[i][0], info_evcc_send_split[i][0])
    stageNum = stageNum + 1
    
    for j in range(1, len(data_length_evcc_send_split[i])):
        #Stage 2
        if(data_length_evcc_send_split[i][j] == 100 and data_length_evcc_send_split[i][j-1] != 100):
            if(checkstage_num(stageNum, 2, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                #Deal with packet loss......
                if(i == 5 and packet_loss_testing_round_5_flag_1 == False):
                    evcc_send_time_anchor_each_round.append(202.792)
                    printinfo(stageNum, 25114, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_5_flag_1 = True
                #Deal with packet loss......
                if(i == 5 and packet_loss_testing_round_5_flag_2 == False):
                    evcc_send_time_anchor_each_round.append(202.920379)
                    printinfo(stageNum, 25124, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_5_flag_2 = True
                #Deal with packet loss......
                if(i == 5 and packet_loss_testing_round_5_flag_3 == False):
                    evcc_send_time_anchor_each_round.append(202.974838)
                    printinfo(stageNum, 25128, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_5_flag_3 = True
                    continue
                
        #Stage 3
        if(data_length_evcc_send_split[i][j] == 189 and data_length_evcc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 3, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                 #Deal with packet loss......
                if(i == 17 and packet_loss_testing_round_17_flag_2 == False):
                    evcc_send_time_anchor_each_round.append(565.406963)
                    printinfo(stageNum, 53149, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_17_flag_2 = True
                
                continue
                
        #Stage 4 
        if(data_length_evcc_send_split[i][j] == 173 and data_length_evcc_send_split[i][j-1] != 173):
            
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 4, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    continue
            else: print("Stage " + str(stageNum) +
                        ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                        ", Data length " + str(data_length_evcc_send_split[i][j]) +
                        ", Info " + str(info_evcc_send_split[i][j]))
        #Stage 5, tricky
        if(data_length_evcc_send_split[i][j] == 173):    
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 5, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    #Deal with packet loss......
                    if(i == 3 and packet_loss_testing_round_3_flag == False):
                        evcc_send_time_anchor_each_round.append(142.582864)
                        printinfo(stageNum, 11788, "Packet Loss")
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                        packet_loss_testing_round_3_flag = True
                        
                    continue
                
            else: print("Stage " + str(stageNum) +
                        ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                        ", Data length " + str(data_length_evcc_send_split[i][j]) +
                        ", Info " + str(info_evcc_send_split[i][j]))
        #Stage 6, tricky
        if(data_length_evcc_send_split[i][j] == 173):
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 6, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    continue
            else: print("Stage " + str(stageNum) +
                        ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                        ", Data length " + str(data_length_evcc_send_split[i][j]) +
                        ", Info " + str(info_evcc_send_split[i][j]))
        #Stage 7
        if(data_length_evcc_send_split[i][j] == 185 and data_length_evcc_send_split[i][j-1] != 185):
            #Deal with packet loss......
            if(i == 17 and packet_loss_testing_round_17_flag == False):
                evcc_send_time_anchor_each_round.append(566.331214)
                printinfo(stageNum, 53479, "Packet Loss")
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                packet_loss_testing_round_17_flag = True
                
            if(checkstage_num(stageNum, 7, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
        #Stage 8
        if(data_length_evcc_send_split[i][j] == 461 and data_length_evcc_send_split[i][j-1] != 461):
            if(checkstage_num(stageNum, 8, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
        #Stage 9
        if(data_length_evcc_send_split[i][j] == 189 and data_length_evcc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 9, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                #Deal with packet loss......
                if(i == 3 and packet_loss_testing_round_3_flag_2 == False):
                    evcc_send_time_anchor_each_round.append(143.257273)
                    printinfo(stageNum, 11989, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_3_flag_2 = True
                
                continue
        #Stage 10
        if(data_length_evcc_send_split[i][j] == 173 and data_length_evcc_send_split[i][j-1] != 173):
            
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 10, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    continue
            else: print("Stage " + str(stageNum) +
                        ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                        ", Data length " + str(data_length_evcc_send_split[i][j]) +
                        ", Info " + str(info_evcc_send_split[i][j]))
                
        if(stageNum == 11): break
        
    evcc_send_time_anchor.append(evcc_send_time_anchor_each_round)

In [236]:
#for debug
print(str(len(evcc_send_time_anchor)) + '\n')
for i in range(0, len(evcc_send_time_anchor)):
    print(len(evcc_send_time_anchor[i]))

20

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [237]:
wierd_packet

{2685: [185, 8],
 5663: [185, 8],
 8867: [185, 8],
 20299: [185, 8],
 25174: [185, 8],
 27470: [173, 3],
 27627: [185, 8],
 29743: [185, 8],
 32013: [185, 8],
 35727: [173, 3],
 35956: [185, 8],
 38125: [173, 3],
 38347: [189, 10],
 41323: [185, 8],
 41452: [189, 10],
 43176: [185, 8],
 43251: [189, 10],
 45113: [185, 8],
 46893: [173, 3],
 47080: [185, 8],
 49125: [185, 8],
 51055: [185, 8],
 53479: [185, 8],
 53488: [185, 8],
 55278: [173, 3],
 55482: [185, 8],
 57309: [185, 8]}

In [238]:
len(wierd_packet)

27

In [239]:
#for debug
for i in range(0, len(evcc_send_time_anchor)):
    print(evcc_send_time_anchor[i][len(evcc_send_time_anchor[i])-1]-evcc_send_time_anchor[i][0])

3.0921360000000035
2.8446099999999888
1.9658930000000083
2.430810000000008
2.3667619999999943
2.2047629999999856
2.43728999999999
1.9556529999999839
2.4874940000000265
2.6971340000000055
2.4248440000000073
2.6730419999999526
1.6886739999999918
2.3817449999999667
2.3508110000000215
2.4727079999999546
1.7218189999999822
2.8941449999999804
2.475257000000056
1.9679419999999936


In [240]:
#Processing secc send
with open('max_secc_send.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    timelist_secc_send = []
    seq_num_secc_send = []
    data_length_secc_send = []
    info_secc_send = []
    frame_num_secc_send = []
    
    for row in rows:
        timelist_secc_send.append(float(row['Time']))
        seq_num_secc_send.append(int(row['Sequence number']))
        data_length_secc_send.append(int(row['Data Length']))
        info_secc_send.append(str(row['Info']))
        frame_num_secc_send.append(int(row['No.']))

In [241]:
#for debug
if(len(timelist_secc_send) != len(seq_num_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(data_length_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(info_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(frame_num_secc_send)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_secc_send))

2464


In [242]:
timelist_secc_send_split = []
seq_num_secc_send_split = []
data_length_secc_send_split = []
info_secc_send_split = []
frame_num_secc_send_split = []

round_start_point = 0
round_end_point = 0

sdpres_index_list = []
#Fetch first packet index of data length = 92 (SDP response) in each round
for i in range(0, len(data_length_secc_send)):
    if(i == 0 and data_length_secc_send[i] == 92):
        sdpres_index_list.append(i)
    if(i > 0 and data_length_secc_send[i] == 92 and data_length_secc_send[i-1] != 92):
        sdpres_index_list.append(i)
sdpres_index_list.append(len(data_length_secc_send)) #Add the end of list index

for i in range(0, len(sdpres_index_list)-1):
        timelist_secc_send_split.append([timelist_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        seq_num_secc_send_split.append([seq_num_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        data_length_secc_send_split.append([data_length_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        info_secc_send_split.append([info_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        frame_num_secc_send_split.append([frame_num_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])

In [243]:
#for debug
if(len(timelist_secc_send_split) != len(seq_num_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(data_length_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(info_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(frame_num_secc_send_split)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_secc_send_split)) #Output # of testing rounds

21


In [ ]:
for i in range(0, len(timelist_secc_send_split)):
    print('i = ' + str(i))
    print(len(timelist_secc_send_split[i]))

In [245]:
#for debug, this is ideally the time of each testing round
for i in range(1, len(timelist_secc_send_split)):
    print(timelist_secc_send_split[i][0]-timelist_secc_send_split[i-1][0])

31.24035899999999
30.81248500000001
29.99349199999999
30.31234600000002
30.208534999999983
30.344866999999994
30.479444
30.113652000000002
31.115973999999994
30.925652000000014
30.023331999999982
30.50526100000002
29.57543099999998
30.007479000000046
30.001979999999946
0.006025000000022374
30.30554699999999
29.279791000000046
30.62495100000001
30.30801299999996


In [246]:
#Catch error. Frame No. 48660
timelist_secc_send_split[15]

[504.557895, 504.558916]

In [247]:
del timelist_secc_send_split[15]
del seq_num_secc_send_split[15]
del data_length_secc_send_split[15]
del info_secc_send_split[15]
del frame_num_secc_send_split[15]

In [248]:
for i in range(0, len(timelist_secc_send_split)):
    print('i = ' + str(i) + ' ', end = '')
    print(len(timelist_secc_send_split[i]))

i = 0 141
i = 1 129
i = 2 121
i = 3 118
i = 4 116
i = 5 89
i = 6 119
i = 7 114
i = 8 136
i = 9 122
i = 10 119
i = 11 128
i = 12 126
i = 13 114
i = 14 122
i = 15 120
i = 16 119
i = 17 128
i = 18 131
i = 19 150


In [249]:
for i in range(1, len(timelist_secc_send_split)):
    print(timelist_secc_send_split[i][0]-timelist_secc_send_split[i-1][0])

31.24035899999999
30.81248500000001
29.99349199999999
30.31234600000002
30.208534999999983
30.344866999999994
30.479444
30.113652000000002
31.115973999999994
30.925652000000014
30.023331999999982
30.50526100000002
29.57543099999998
30.007479000000046
30.00800499999997
30.30554699999999
29.279791000000046
30.62495100000001
30.30801299999996


In [250]:
print(len(timelist_secc_send_split))

20


In [ ]:
#Set secc time anchor in each round per stage
secc_send_time_anchor = []
#data_length_in_each_stage = [92, 173, 157, 189, 253, 173, 189, 173, 493, 173]
wierd_packet_secc = {}
packet_loss_testing_round_5_flag = False
packet_loss_testing_round_6_flag = False
packet_loss_testing_round_9_flag = False
packet_loss_testing_round_13_flag = False
packet_loss_testing_round_14_flag = False
packet_loss_testing_round_17_flag = False

def printinfo(stageNum, frameNum, info_string):
    print("-----------------------------------------")
    print("Stage " + str(stageNum) + " finish. No. " + str(frameNum) + ", Info " + str(info_string))

def print_time_difference(secc_send_time_anchor_each_round):
    print("Time difference: " + str(secc_send_time_anchor_each_round[len(secc_send_time_anchor_each_round)-1]-
                          secc_send_time_anchor_each_round[len(secc_send_time_anchor_each_round)-2]))
    
def checkstage_num(stageNum, valid_stageNum, info_string, time_string, datalen, seqnum, frameNum):
    if(stageNum == valid_stageNum):
        check_wield_packet_list(frameNum)
        return True
    else:
        #debug
        print("\nWhat is this packet? Data Length = " + str(datalen) + " No. " + str(frameNum))
        print("Time " + str(time_string) + " #Seq " + str(seqnum) + " Info " + str(info_string) + "\n")
        wierd_packet_secc[frameNum] = [datalen, stageNum]
        return False

def check_wield_packet_list(frameNum):
    if(wierd_packet_secc.get(frameNum) != None):
        del wierd_packet_secc[frameNum]
        print("\nNo. " + str(frameNum) + " has been found. Delete from wierd list.\n")
    
for i in range(0, len(data_length_secc_send_split)):
    print("\n-------- Test Round " + str(i+1) + " --------")
    #Stage 1
    secc_send_time_anchor_each_round = [timelist_secc_send_split[i][0]]
    stageNum = 1
    printinfo(stageNum, frame_num_secc_send_split[i][0], info_secc_send_split[i][0])
    stageNum = stageNum + 1
    
    #Deal with packet loss......
    if(i == 5 and packet_loss_testing_round_5_flag == False):
        secc_send_time_anchor_each_round.append(202.710589)
        printinfo(stageNum, 25112, "Packet Loss")
        print_time_difference(secc_send_time_anchor_each_round)
        stageNum = stageNum + 1
        
        secc_send_time_anchor_each_round.append(202.799629)
        printinfo(stageNum, 25116, "Packet Loss")
        print_time_difference(secc_send_time_anchor_each_round)
        stageNum = stageNum + 1
        
        secc_send_time_anchor_each_round.append(202.926506)
        printinfo(stageNum, 25126, "Packet Loss")
        print_time_difference(secc_send_time_anchor_each_round)
        stageNum = stageNum + 1
        
        secc_send_time_anchor_each_round.append(202.981442)
        printinfo(stageNum, 25130, "Packet Loss")
        print_time_difference(secc_send_time_anchor_each_round)
        stageNum = stageNum + 1
        
        packet_loss_testing_round_5_flag = True
    
    #Deal with packet loss......
    if(i == 13 and packet_loss_testing_round_13_flag == False):
        secc_send_time_anchor_each_round.append(445.566241)
        printinfo(stageNum, 44926, "Packet Loss")
        print_time_difference(secc_send_time_anchor_each_round)
        stageNum = stageNum + 1
        packet_loss_testing_round_13_flag = True
        
    #Deal with packet loss......
    if(i == 14 and packet_loss_testing_round_14_flag == False):
        secc_send_time_anchor_each_round.append(475.490112)
        printinfo(stageNum, 46918, "Packet Loss")
        print_time_difference(secc_send_time_anchor_each_round)
        stageNum = stageNum + 1
        packet_loss_testing_round_14_flag = True
    
    for j in range(1, len(data_length_secc_send_split[i])):
        #Stage 2
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 2, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    continue
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 3
        if(data_length_secc_send_split[i][j] == 157 and data_length_secc_send_split[i][j-1] != 157):
            if(checkstage_num(stageNum, 3, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
        #Stage 4
        if(data_length_secc_send_split[i][j] == 189 and data_length_secc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 4, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                #Deal with packet loss......
                if(i == 6 and packet_loss_testing_round_6_flag == False):
                    secc_send_time_anchor_each_round.append(233.058599)
                    printinfo(stageNum, 27570, "Packet Loss")
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_6_flag = True
                    
                continue
        #Stage 5
        if(data_length_secc_send_split[i][j] == 253 and data_length_secc_send_split[i][j-1] != 253):
            if(checkstage_num(stageNum, 5, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                #Deal with packet loss......
                if(i == 9 and packet_loss_testing_round_9_flag == False):
                    secc_send_time_anchor_each_round.append(325.166882)
                    printinfo(stageNum, 35905, "Packet Loss")
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_9_flag = True
                
                #Deal with packet loss......
                if(i == 17 and packet_loss_testing_round_17_flag == False):
                    secc_send_time_anchor_each_round.append(565.802852)
                    printinfo(stageNum, 53285, "Packet Loss")
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_17_flag = True
                
                continue
        #Stage 6
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 6, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    continue
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 7
        if(data_length_secc_send_split[i][j] == 189 and data_length_secc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 7, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
        #Stage 8
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 8, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    continue
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 9
        if(data_length_secc_send_split[i][j] == 493 and data_length_secc_send_split[i][j-1] != 493):
            if(checkstage_num(stageNum, 9, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
        #Stage 10
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 10, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    continue
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        
        if(stageNum == 11): break
        
    secc_send_time_anchor.append(secc_send_time_anchor_each_round)

In [252]:
#for debug
print(str(len(secc_send_time_anchor)) + '\n')
for i in range(0, len(secc_send_time_anchor)):
    print(len(secc_send_time_anchor[i]))

20

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [253]:
#for debug
for i in range(0, len(secc_send_time_anchor)):
    print(secc_send_time_anchor[i][len(secc_send_time_anchor[i])-1]-secc_send_time_anchor[i][0])

3.0079819999999984
2.8719090000000023
1.9112329999999957
2.3272750000000144
2.3059009999999773
2.2136859999999956
2.3172500000000014
1.9016450000000304
2.4609159999999974
2.783100999999988
2.331598999999983
2.6078370000000177
1.6572009999999864
2.391020000000026
2.3249339999999847
2.395364000000029
1.660641000000055
2.8569649999999456
2.4298369999999068
1.87462000000005


In [254]:
secc_send_time_anchor[0]

[48.897606,
 49.961497,
 50.152844,
 50.337895,
 50.377603,
 51.161093,
 51.515046,
 51.692992,
 51.74107,
 51.905588]

In [ ]:
#BLOCK ACK processing
#Time evcc sends blockACK after secc send
evcc_blockAck_time_anchor = []

blockack_index = 0
for i in range(0, len(secc_send_time_anchor)):
    evcc_blockAck_time_anchor_in_each_round = []
    
    for j in range(0, len(secc_send_time_anchor[i])):
        while(timelist_block_ack[blockack_index] < secc_send_time_anchor[i][j]):
            blockack_index = blockack_index + 1
        evcc_blockAck_time_anchor_in_each_round.append(timelist_block_ack[blockack_index])
        print("ACK time difference = " + str(
        timelist_block_ack[blockack_index] - secc_send_time_anchor[i][j]))
    
    evcc_blockAck_time_anchor.append(evcc_blockAck_time_anchor_in_each_round)

In [272]:
#SECC Packet Loss dict
#secc_packet_loss_dict[i] = {stageNum}
# i = 5, j = 0 is packet loss of evcc BLOCK ACK
# i = 3, j = 9 is packet loss of evcc BLOCK ACK
secc_packet_loss_dict = {}
secc_packet_loss_dict[3] = {9}
secc_packet_loss_dict[5] = {0, 1, 2, 3, 4}
secc_packet_loss_dict[6] = {4}
secc_packet_loss_dict[9] = {5}
secc_packet_loss_dict[13] = {1}
secc_packet_loss_dict[14] = {1}
secc_packet_loss_dict[17] = {5}

#for i in secc_packet_loss_dict.get(5): print(i)
#7 in secc_packet_loss_dict.get(5)

In [273]:
#Calculate the avg. block ack latency
sum_latency = 0
cnt = 0
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) == None):
            sum_latency = sum_latency + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
            cnt = cnt + 1
        elif(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)): continue
            else:
                sum_latency = sum_latency + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
                cnt = cnt + 1
        else: continue
avg_blockAck_latency = sum_latency/cnt
print("Avg Block Ack latency = " + str(avg_blockAck_latency) + " s")
print(cnt)

Avg Block Ack latency = 0.007726566137568847 s
189


In [274]:
#For curiousity
#Calculate the avg. block ack latency of the appending packet loss packets
sum_latency_2 = 0
cnt = 0
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)):
                sum_latency_2 = sum_latency_2 + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
                cnt = cnt + 1
        else: continue
avg_blockAck_latency_2 = sum_latency_2/cnt
print("Avg Block Ack latency = " + str(avg_blockAck_latency_2) + " s")
print(cnt)

Avg Block Ack latency = 0.18464700000000211 s
11


In [275]:
#For debug. Can see testing round 6 (i = 5) has longer block ack latency due to packet loss.
# I only append the secc send time, but didn't append the block time.
from statistics import mean

for i in range(0, len(evcc_blockAck_time_anchor)):
    print("i = " + str(i) + ": " + str(mean(evcc_blockAck_time_anchor[i])-mean(secc_send_time_anchor[i])))

i = 0: 0.009068800000001431
i = 1: 0.007825699999997937
i = 2: 0.00894599999999457
i = 3: 0.01699550000000727
i = 4: 0.009264399999977968
i = 5: 0.1695633000000214
i = 6: 0.011674600000020519
i = 7: 0.011354400000016085
i = 8: 0.0061952000000360385
i = 9: 0.0357298000000128
i = 10: 0.007326200000022709
i = 11: 0.003543000000036045
i = 12: 0.004891700000030141
i = 13: 0.007610999999997148
i = 14: 0.006605499999977837
i = 15: 0.005972399999961908
i = 16: 0.005601600000090912
i = 17: 0.011716200000023491
i = 18: 0.0033443000000943357
i = 19: 0.005914200000006531


In [276]:
#For debug
for j in range(0, 10):
    print(evcc_blockAck_time_anchor[0][j]-secc_send_time_anchor[0][j])

0.0008429999999961524
0.0010869999999982838
0.0014240000000000919
0.0020569999999935362
0.0009629999999987149
0.013097000000001913
0.029555000000001996
0.007459000000004323
0.025390999999999053
0.008811999999998932


In [277]:
#Use the avg. block ack latency for the lost secc sent packets
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)):
                evcc_blockAck_time_anchor[i][j] = secc_send_time_anchor[i][j] + avg_blockAck_latency
                print("i = " + str(i))
                print("j = " + str(j))
        else: continue

i = 3
j = 9
i = 5
j = 0
i = 5
j = 1
i = 5
j = 2
i = 5
j = 3
i = 5
j = 4
i = 6
j = 4
i = 9
j = 5
i = 13
j = 1
i = 14
j = 1
i = 17
j = 5


In [278]:
avg_blockAck_latency

0.007726566137568847

In [279]:
len(evcc_blockAck_time_anchor)

20

In [280]:
for i in range(0, len(evcc_blockAck_time_anchor)):
    print(len(evcc_blockAck_time_anchor[i]))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [281]:
#for debug check wierd value
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_send_time_anchor[i][j] - evcc_send_time_anchor[i][j] < 0):
            print("i = " + str(i) + " j = " + str(j))
        if(evcc_blockAck_time_anchor[i][j] - evcc_send_time_anchor[i][j] < 0):
            print("i = " + str(i) + " j = " + str(j))

In [282]:
secc_send_time_anchor[17]

[564.149258,
 565.174129,
 565.274518,
 565.420973,
 565.450665,
 565.802852,
 566.4488,
 566.834902,
 566.870918,
 567.006223]

In [283]:
#For debug
#secc send is based on 802.11 capture file (not 802.11_TCP combined file)
i = 3
for j in range(0, 10):
    print("Stage " + str(j+1))
    print(evcc_send_time_anchor[i][j])
    print(secc_send_time_anchor[i][j])
    print(evcc_blockAck_time_anchor[i][j])
    print(evcc_blockAck_time_anchor[i][j]-evcc_send_time_anchor[i][j])
    print("\n")

Stage 1
140.826463
140.943942
140.965527
0.13906400000001895


Stage 2
141.56852
141.990122
141.992905
0.4243850000000009


Stage 3
142.11107
142.173171
142.173226
0.062155999999987444


Stage 4
142.322301
142.381302
142.384616
0.06231499999998391


Stage 5
142.396109
142.430599
142.440971
0.04486199999999485


Stage 6
142.582864
142.593974
142.608126
0.025261999999997897


Stage 7
142.842789
142.894494
142.904
0.061210999999985916


Stage 8
143.043018
143.081551
143.099235
0.056217000000003736


Stage 9
143.120781
143.144603
143.153464
0.03268300000002


Stage 10
143.257273
143.271217
143.27894356613757
0.021670566137572678




In [284]:
path = 'max_80211_pcap_processing.txt'
f = open(path, 'w')
for i in range(0, len(evcc_send_time_anchor)):
    for j in range(0, len(evcc_send_time_anchor[i])):
        print((evcc_blockAck_time_anchor[i][j] - evcc_send_time_anchor[i][j])*1000, file=f, end="")
        f.writelines('\t')
    f.writelines('\t')
    print((evcc_blockAck_time_anchor[i][1] - evcc_send_time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    print((evcc_blockAck_time_anchor[i][len(evcc_blockAck_time_anchor[i])-1] - evcc_send_time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    f.writelines('20')
    f.writelines('\t')
    f.writelines('0')
    f.writelines('\t')
    f.writelines('\n')
f.close()